In [1]:
import pandas as pd

In [2]:
path = './RAWDATA_Ward_Scenarios.xlsx'
df_wards = pd.read_excel(path)

In [3]:
df_wards.head()

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use
0,ATC,ADD L2,Day surgery,35,excl.,0,excl.,0,excl.,0,NaN
1,ATC,ADD L4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients
2,ATC,ADD M4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients
3,ATC,ADD L5,GIM,32,G,32,G,32,R,22,Elective surgery patients
4,ATC,ADD M5,GIM,32,G,32,G,32,R,22,Elective surgery patients


In [4]:
df_wards['AB_change'] = df_wards.A_color != df_wards.B_color
df_wards['BC_change'] = df_wards.B_color != df_wards.C_color
df_wards.head()

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use,AB_change,BC_change
0,ATC,ADD L2,Day surgery,35,excl.,0,excl.,0,excl.,0,NaN,False,False
1,ATC,ADD L4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients,False,False
2,ATC,ADD M4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients,False,False
3,ATC,ADD L5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True
4,ATC,ADD M5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True


In [5]:
import numpy as np

def _shuffled_range(n):
    """
    Generates a randomly shuffled range from 0 to n-1
    """
    shuffled_range = np.arange(n)
    np.random.shuffle(shuffled_range)
    return shuffled_range

In [6]:
df_wards['AB_change_no'] = None

change_no = _shuffled_range(len(df_wards[df_wards.AB_change]))
df_wards.loc[df_wards.AB_change, 'AB_change_no'] = change_no

df_wards[df_wards.AB_change]

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use,AB_change,BC_change,AB_change_no
14,F&G,ADD F4,GIM,19,G,19,R,13,R,13,Renal,True,False,0
15,F&G,ADD G4,GIM,21,G,21,R,16,R,16,Hepatology,True,False,5
17,F&G,ADD F6,GIM,27,G,27,A,9,A,9,Gastro,True,False,2
18,F&G,ADD G6,GIM,26,G,26,R,21,R,21,DME,True,False,6
25,C&D,ADD C5,GIM,23,RG,23,R,23,R,23,Renal,True,False,3
26,C&D,ADD D5,GIM,30,A,13,R,24,R,24,DME,True,False,4
28,C&D,ADD D6,GIM,31,A,15,R,27,R,27,Neuro/Stroke/Neurosurgery/Gastro,True,False,1
32,C&D,ADD D8,GIM,36,DECANT,0,A,15,A,15,DECANT ward (fire safety works),True,False,7


In [7]:
df_wards['BC_change_no'] = None

change_no = _shuffled_range(len(df_wards[df_wards.BC_change]))
df_wards.loc[df_wards.BC_change, 'BC_change_no'] = change_no

df_wards[df_wards.BC_change]

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use,AB_change,BC_change,AB_change_no,BC_change_no
3,ATC,ADD L5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True,None,8
4,ATC,ADD M5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True,None,10
5,Rosie,ROS DAPHNE,GIM,14,G,14,G,14,R,9,Gynae/Gynae Onc/Breast,False,True,None,5
6,J/K/N,ADD K3/CCU,GIM (20); ICU (10),30,G,30,G,30,R,21,Cardiology,False,True,None,1
8,J/K/N,ADD N3,GIM,25,G,25,G,25,R,17,Respiratory,False,True,None,4
9,J/K/N,ADD J2,GIM,21,G,21,G,21,R,14,Major trauma rehab,False,True,None,2
13,F&G,ADD G3,GIM,27,G,27,G,27,R,18,Diabetes,False,True,None,7
16,F&G,ADD F/G5,GIM,34,G,34,G,34,R,23,Transplant,False,True,None,6
23,C&D,ADD C4,GIM,29,G,29,G,29,R,20,DME,False,True,None,3
33,C&D,ADD C9,GIM,8,G,8,G,8,R,5,Oncology,False,True,None,9


In [8]:
df_wards.groupby(by='A_color').agg(No_Wards=('A_no_beds', 'count'), No_Beds=('A_no_beds', 'sum'))

,No_Wards,No_Beds
A_color,,
A,6,95
DECANT,1,0
G,23,550
R,4,84
RG,1,23
excl.,12,0


In [9]:
df_wards.head()

,block,id,type,max_no_beds,A_color,A_no_beds,B_color,B_no_beds,C_color,C_no_beds,normal_use,AB_change,BC_change,AB_change_no,BC_change_no
0,ATC,ADD L2,Day surgery,35,excl.,0,excl.,0,excl.,0,NaN,False,False,None,None
1,ATC,ADD L4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients,False,False,None,None
2,ATC,ADD M4,GIM,32,G,32,G,32,G,32,Non-elective surgery patients,False,False,None,None
3,ATC,ADD L5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True,None,8
4,ATC,ADD M5,GIM,32,G,32,G,32,R,22,Elective surgery patients,False,True,None,10


In [10]:
df_melt = df_wards.melt(id_vars=['block', 'id', 'max_no_beds', 'A_no_beds', 'B_no_beds', 'C_no_beds', 'AB_change', 'BC_change'], 
              value_vars=['A_color', 'B_color', 'C_color'], 
              var_name='scenario',
              value_name='color'
             )

In [11]:
df_melt.loc[:,'scenario'] = df_melt.scenario.apply(lambda s: s[0])

In [12]:
df_melt

,block,id,max_no_beds,A_no_beds,B_no_beds,C_no_beds,AB_change,BC_change,scenario,color
0,ATC,ADD L2,35,0,0,0,False,False,A,excl.
1,ATC,ADD L4,32,32,32,32,False,False,A,G
2,ATC,ADD M4,32,32,32,32,False,False,A,G
3,ATC,ADD L5,32,32,32,22,False,True,A,G
4,ATC,ADD M5,32,32,32,22,False,True,A,G
...,...,...,...,...,...,...,...,...,...,...
136,R,ADD LEWIN (REHAB),26,26,26,26,False,False,C,G
137,A,ADD A2 (NCCU),23,0,0,0,False,False,C,excl.
138,A,ADD A3,11,8,8,8,False,False,C,G
139,A,ADD A4,26,26,26,26,False,False,C,G


In [22]:
df_melt['no_beds'] = None
for scenario in df_melt.scenario.unique():
    col = scenario + '_no_beds'
    df_melt.loc[df_melt.scenario == scenario,'no_beds'] = df_melt.loc[df_melt.scenario == scenario,col]

In [25]:
df_melt = df_melt.drop(columns=['A_no_beds', 'B_no_beds', 'C_no_beds'])

In [28]:
df_melt[df_melt.AB_change | df_melt.BC_change].sort_values(by=['id', 'scenario'])

,block,id,max_no_beds,AB_change,BC_change,scenario,color,no_beds
35,C&D,ADD C10,16,False,True,A,G,13
82,C&D,ADD C10,16,False,True,B,G,13
129,C&D,ADD C10,16,False,True,C,R,9
23,C&D,ADD C4,29,False,True,A,G,29
70,C&D,ADD C4,29,False,True,B,G,29
117,C&D,ADD C4,29,False,True,C,R,20
25,C&D,ADD C5,23,True,False,A,RG,23
72,C&D,ADD C5,23,True,False,B,R,23
119,C&D,ADD C5,23,True,False,C,R,23
33,C&D,ADD C9,8,False,True,A,G,8


In [29]:
df_melt.pivot(index='id', columns='scenario', values=['color', 'no_beds'])

color               no_beds        
scenario                    A      B      C       A   B   C
id                                                         
ADD A2 (NCCU)           excl.  excl.  excl.       0   0   0
ADD A3                      G      G      G       8   8   8
ADD A4                      G      G      G      26  26  26
ADD A5                      G      G      G      28  28  28
ADD C10                     G      G      R      13  13   9
ADD C2                  excl.  excl.  excl.       0   0   0
ADD C3                  excl.  excl.  excl.       0   0   0
ADD C4                      G      G      R      29  29  20
ADD C5                     RG      R      R      23  23  23
ADD C6                      R      R      R      24  24  24
ADD C7                      R      R      R      23  23  23
ADD C8                      A      A      A      19  19  19
ADD C9                      G      G      R       8   8   5
ADD D10                     R      R      R      11  11  11
ADD D2                  excl.  excl.  excl.       0   0   0
ADD D3                  excl.  excl.  excl.       0   0   0
ADD D4                  excl.  excl.  excl.       0   0   0
ADD D5                      A      R      R      13  24  24
ADD D6                      A      R      R      15  27  27
ADD D7                      A      A      A      13  13  13
ADD D8                 DECANT      A      A       0  15  15
ADD D9                      A      A      A      13  13  13
ADD EAU2                excl.  excl.  excl.       0   0   0
ADD EAU3                excl.  excl.  excl.       0   0   0
ADD EAU4                    G      G      R      26  26  18
ADD EAU5 (MED S-STAY)       R      R      R      26  26  26
ADD F/G5                    G      G      R      34  34  23
ADD F2 (PICU/HDU)       excl.  excl.  excl.       0   0   0
ADD F3                  excl.  excl.  excl.       0   0   0
ADD F4                      G      R      R      19  13  13
ADD F6                      G      A      A      27   9   9
ADD G3                      G      G      R      27  27  18
ADD G4                      G      R      R      21  16  16
ADD G6                      G      R      R      26  21  21
ADD J2                      G      G      R      21  21  14
ADD J3                  excl.  excl.  excl.       0   0   0
ADD K3/CCU                  G      G      R      30  30  21
ADD L2                  excl.  excl.  excl.       0   0   0
ADD L4                      G      G      G      32  32  32
ADD L5                      G      G      R      32  32  22
ADD LEWIN (REHAB)           G      G      G      26  26  26
ADD M4                      G      G      G      32  32  32
ADD M5                      G      G      R      32  32  22
ADD N2                      A      A      A      22  22  22
ADD N3                      G      G      R      25  25  17
ADD R2                      G      G      G      14  14  14
ROS DAPHNE                  G      G      R      14  14   9